In [1]:
from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout, BatchNormalization, UpSampling2D,Concatenate, Activation,Add
from keras import backend as K
from keras import optimizers
from keras.applications.resnet50 import ResNet50



Using TensorFlow backend.


In [2]:
resnet50_model = ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(197, 197, 3), pooling=None)

In [3]:
resnet50_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 197, 197, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 99, 99, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 99, 99, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 99, 99, 64)   0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [6]:
resnet50_model.layers[3].get_config()
# resnet50_model.layers[36].get_config()
# resnet50_model.layers[78].get_config()
# resnet50_model.layers[140].get_config()
# resnet50_model.layers[172].get_config()

{'activation': 'relu', 'name': 'activation_1', 'trainable': True}

In [7]:
# resnet50_model.layers[3].get_config()
resnet50_model.layers[36].get_config()
# resnet50_model.layers[78].get_config()
# resnet50_model.layers[140].get_config()
# resnet50_model.layers[172].get_config()

{'activation': 'relu', 'name': 'activation_10', 'trainable': True}

In [8]:
# resnet50_model.layers[3].get_config()
# resnet50_model.layers[36].get_config()
resnet50_model.layers[78].get_config()
# resnet50_model.layers[140].get_config()
# resnet50_model.layers[172].get_config()

{'activation': 'relu', 'name': 'activation_22', 'trainable': True}

In [9]:
# resnet50_model.layers[3].get_config()
# resnet50_model.layers[36].get_config()
# resnet50_model.layers[78].get_config()
resnet50_model.layers[140].get_config()
# resnet50_model.layers[172].get_config()

{'activation': 'relu', 'name': 'activation_40', 'trainable': True}

In [10]:
# resnet50_model.layers[3].get_config()
# resnet50_model.layers[36].get_config()
# resnet50_model.layers[78].get_config()
# resnet50_model.layers[140].get_config()
resnet50_model.layers[172].get_config()

{'activation': 'relu', 'name': 'activation_49', 'trainable': True}

In [22]:
# from IPython.display import Image, display, SVG
# from keras.utils.vis_utils import model_to_dot

# # Show the model in ipython notebook
# figure = SVG(model_to_dot(resnet50_model, show_shapes=True, show_layer_names=True).create(prog='dot', format='svg'))
# display(figure)

In [23]:
# from keras.utils.vis_utils import plot_model
# plot_model(resnet50_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [9]:
class ResNet:
    @staticmethod
    def residual_module(data, K, stride, chanDim, red=False, reg=0.0001, bnEps=2e-5, bnMom=0.9):
        # the shortcut branch of the ResNet module should be
        # initialize as the input (identity) data
        shortcut = data
        
        # the first block of the ResNet module are the 1x1 CONVs
        bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps,
        momentum=bnMom)(data)
        act1 = Activation("relu")(bn1)
        conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False,kernel_regularizer=l2(reg))(act1)
        
        # the second block of the ResNet module are the 3x3 CONVs
        bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps,
        momentum=bnMom)(conv1)
        act2 = Activation("relu")(bn2)
        conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride,
        padding="same", use_bias=False,
        kernel_regularizer=l2(reg))(act2)

        # the third block of the ResNet module is another set of 1x1
        # CONVs
        bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps,
        momentum=bnMom)(conv2)
        act3 = Activation("relu")(bn3)
        conv3 = Conv2D(K, (1, 1), use_bias=False,
        kernel_regularizer=l2(reg))(act3)

        # add together the shortcut and the final CONV
        x = add([conv3, shortcut])
        # return the addition as the output of the ResNet module
        return x

    @staticmethod
    def build(width, height, depth, classes, stages, filters,reg=0.0001, bnEps=2e-5, bnMom=0.9, dataset="cifar"):
        # initialize the input shape to be "channels last" and the
        # channels dimension itself
        inputShape = (height, width, depth)
        chanDim = -1
        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1

        # set the input and apply BN
        inputs = Input(shape=inputShape)
        x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(inputs)
        # check if we are utilizing the CIFAR dataset
        if dataset == "cifar":
            # apply a single CONV layer
            x = Conv2D(filters[0], (3, 3), use_bias=False, padding="same", kernel_regularizer=l2(reg))(x)
        # loop over the number of stages
        for i in range(0, len(stages)):
            # initialize the stride, then apply a residual module
            # used to reduce the spatial size of the input volume
            stride = (1, 1) if i == 0 else (2, 2)
            x = ResNet.residual_module(x, filters[i + 1], stride, chanDim, red=True, bnEps=bnEps, bnMom=bnMom)
        
        # loop over the number of layers in the stage
        for j in range(0, stages[i] - 1):
            # apply a ResNet module
            x = ResNet.residual_module(x, filters[i + 1], (1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)
            
        # apply BN => ACT => POOL
        x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
        x = Activation("relu")(x)
        x = AveragePooling2D((8, 8))(x)

        #softmax classifier
        x = Flatten()(x)
        x = Dense(classes, kernel_regularizer=l2(reg))(x)
        x = Activation("softmax")(x)
        
        # create the model
        model = Model(inputs, x, name="resnet")
        
        # return the constructed network architecture
        return model



In [24]:
# #build(width, height, depth, classes, stages, filters
# model = ResNet.build(32, 32 ,3, 10, (9, 9, 9), (64, 64, 128, 256), reg=0.0005)


# Defining the U-Net with Resnet on the bottom

In [4]:
# Creating dictionary that maps layer names to the layers
layers = dict([(layer.name, layer) for layer in resnet50_model.layers])


In [29]:
resnet50_model.get_config()


{'input_layers': [['input_4', 0, 0]],
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 197, 197, 3),
    'dtype': 'float32',
    'name': 'input_4',
    'sparse': False},
   'inbound_nodes': [],
   'name': 'input_4'},
  {'class_name': 'Conv2D',
   'config': {'activation': 'linear',
    'activity_regularizer': None,
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'filters': 64,
    'kernel_constraint': None,
    'kernel_initializer': {'class_name': 'VarianceScaling',
     'config': {'distribution': 'uniform',
      'mode': 'fan_avg',
      'scale': 1.0,
      'seed': None}},
    'kernel_regularizer': None,
    'kernel_size': (7, 7),
    'name': 'conv1',
    'padding': 'same',
    'strides': (2, 2),
    'trainable': True,
    'use_bias': True},
   'inbound_nodes': [[['input_4', 0, 0, {}]]],
   'name': 'conv1'},
  {

In [30]:
# resnet_top = layers['activation_196'].output

# # Now getting bottom layers for multi-scale skip-layers
# block1_conv2 = layers['activation_187'].output
# block2_conv2 = layers['activation_169'].output
# block3_conv3 = layers['activation_157'].output
# block4_conv3 = layers['activation_148'].output


In [31]:
resnet50_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 197, 197, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 99, 99, 64)   9472        input_4[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 99, 99, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_148 (Activation)     (None, 99, 99, 64)   0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [5]:
resnet50_model.layers.pop()

In [9]:
resnet50_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 197, 197, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 99, 99, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 99, 99, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 99, 99, 64)   0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [35]:
layers

{'activation_148': <keras.layers.core.Activation at 0x7f1befe36fd0>,
 'activation_149': <keras.layers.core.Activation at 0x7f1bef82dbe0>,
 'activation_150': <keras.layers.core.Activation at 0x7f1bec7706d8>,
 'activation_151': <keras.layers.core.Activation at 0x7f1bec6a1208>,
 'activation_152': <keras.layers.core.Activation at 0x7f1bec5ad208>,
 'activation_153': <keras.layers.core.Activation at 0x7f1bec56f860>,
 'activation_154': <keras.layers.core.Activation at 0x7f1bec4bb710>,
 'activation_155': <keras.layers.core.Activation at 0x7f1bec4767b8>,
 'activation_156': <keras.layers.core.Activation at 0x7f1bec3fabe0>,
 'activation_157': <keras.layers.core.Activation at 0x7f1bec340438>,
 'activation_158': <keras.layers.core.Activation at 0x7f1bec2ff160>,
 'activation_159': <keras.layers.core.Activation at 0x7f1bec2818d0>,
 'activation_160': <keras.layers.core.Activation at 0x7f1bec13bcc0>,
 'activation_161': <keras.layers.core.Activation at 0x7f1bec1172e8>,
 'activation_162': <keras.layers.c

In [25]:
i=0
for layer in resnet50_model.layers:
    i=i+1

In [26]:
i

174

In [19]:
from keras import applications
vgg_model = applications.VGG16(weights='imagenet', input_shape=(224, 224, 3), include_top=False)

In [20]:
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [42]:
convm = layers['activation_196'].output

# Now getting bottom layers for multi-scale skip-layers
conv4 = layers['activation_187'].output
conv3 = layers['activation_169'].output
conv2 = layers['activation_157'].output
conv1 = layers['activation_148'].output

In [ ]:
# # Build model for 197 input. The problem was upscale
# def build_model(input_layer, start_neurons, DropoutRatio = 0.5):
#     # 101 -> 50
#     conv1 = Conv2D(start_neurons * 1, (7, 7), strides=(2, 2), activation=None, padding="same")(input_layer)
#     conv1 = residual_block(conv1,start_neurons * 1)
#     conv1 = residual_block(conv1,start_neurons * 1, True)
#     pool1 = MaxPooling2D((2, 2))(conv1)
#     pool1 = Dropout(DropoutRatio/2)(pool1)

#     # 50 -> 25
#     conv2 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(pool1)
#     conv2 = residual_block(conv2,start_neurons * 4)
#     conv2 = residual_block(conv2,start_neurons * 4, True)
#     pool2 = MaxPooling2D((3, 3),strides=(2, 2), padding="same")(conv2)
#     pool2 = Dropout(DropoutRatio)(pool2)

#     # 25 -> 12
#     conv3 = Conv2D(start_neurons * 8, (3, 3),  activation=None, padding="same")(pool2)
#     conv3 = residual_block(conv3,start_neurons * 8)
#     conv3 = residual_block(conv3,start_neurons * 8, True)
#     pool3 = MaxPooling2D((3, 3),strides=(2, 2), padding="same")(conv3)
#     pool3 = Dropout(DropoutRatio)(pool3)

#     # 12 -> 6
#     conv4 = Conv2D(start_neurons * 16, (3, 3),  activation=None, padding="same")(pool3)
#     conv4 = residual_block(conv4,start_neurons * 16)
#     conv4 = residual_block(conv4,start_neurons * 16, True)
#     pool4 = MaxPooling2D((3, 3),strides=(2, 2), padding="same")(conv4)
#     pool4 = Dropout(DropoutRatio)(pool4)

#     # Middle
#     convm = Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same")(pool4)
#     convm = residual_block(convm,start_neurons * 32)
#     convm = residual_block(convm,start_neurons * 32, True)
    
# #     # 6 -> 12
#     deconv4 = Conv2DTranspose(start_neurons * 16, (7, 7), strides=(1, 1), padding="valid")(convm)
#     uconv4 = concatenate([deconv4, conv4])
#     uconv4 = Dropout(DropoutRatio)(uconv4)
    
#     uconv4 = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(uconv4)
#     uconv4 = residual_block(uconv4,start_neurons * 16)
#     uconv4 = residual_block(uconv4,start_neurons * 16, True)
    
#     # 12 -> 25
#     deconv3 = Conv2DTranspose(start_neurons * 8, (13, 13), strides=(1, 1), padding="valid")(uconv4)
#     uconv3 = concatenate([deconv3, conv3])    
#     uconv3 = Dropout(DropoutRatio)(uconv3)
    
#     uconv3 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv3)
#     uconv3 = residual_block(uconv3,start_neurons * 8)
#     uconv3 = residual_block(uconv3,start_neurons * 8, True)

#     # 25 -> 50
#     deconv2 = Conv2DTranspose(start_neurons * 4, (25, 25), strides=(1, 1), padding="valid")(uconv3)
#     uconv2 = concatenate([deconv2, conv2])
        
#     uconv2 = Dropout(DropoutRatio)(uconv2)
#     uconv2 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv2)
#     uconv2 = residual_block(uconv2,start_neurons * 4)
#     uconv2 = residual_block(uconv2,start_neurons * 4, True)
    
#     # 50 -> 101
#     #deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
#     deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="valid")(uconv2)
#     uconv1 = concatenate([deconv1, conv1])
    
#     uconv1 = Dropout(DropoutRatio)(uconv1)
#     uconv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv1)
#     uconv1 = residual_block(uconv1,start_neurons * 1)
#     uconv1 = residual_block(uconv1,start_neurons * 1, True)
    
#      # Extra step
#     #deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
#     deconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="valid")(uconv1)
#     uconv0 = concatenate([deconv0, conv0])
    
#     uconv0 = Dropout(DropoutRatio)(uconv0)
#     uconv0 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
#     uconv0 = residual_block(uconv0,start_neurons * 1)
#     uconv0 = residual_block(uconv0,start_neurons * 1, True)   
    
    
#     #uconv1 = Dropout(DropoutRatio/2)(uconv1)
#     #output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
#     output_layer_noActi = Conv2D(1, (1,1), padding="same", activation=None)(uconv0)
#     output_layer =  Activation('sigmoid')(output_layer_noActi)
    
#     return output_layer

# Build Model

In [2]:
def BatchActivate(x):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    if activation == True:
        x = BatchActivate(x)
    return x

def residual_block(blockInput, num_filters=64, batch_activate = False):
    x = BatchActivate(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    if batch_activate:
        x = BatchActivate(x)
    return x

In [27]:
#Build model for 197 input. The problem was upscale
def build_model(input_layer, start_neurons, base_model, DropoutRatio = 0.5):

    # Get Resnet
    #resnet50_model = ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape= input_layer, pooling=None)
    
    base_model.layers.pop()
    
    for layer in base_model.layers[:173]:
        layer.trainable = False

#     # Creating dictionary that maps layer names to the layers
#     layers = dict([(layer.name, layer) for layer in base_model.layers])

#     conv1 = layers['activation_148'].output
#     conv2 = layers['activation_157'].output
#     conv3 = layers['activation_169'].output
#     conv4 = layers['activation_187'].output
#     convm = layers['activation_196'].output
    
    
    conv1 = base_model.layers[3].output
    conv2 = base_model.layers[36].output
    conv3 = base_model.layers[78].output
    conv4 = base_model.layers[140].output
    convm = base_model.layers[172].output

    # 6 -> 12
    deconv4 = Conv2DTranspose(start_neurons * 16, (7, 7), strides=(1, 1), padding="valid")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(DropoutRatio)(uconv4)
    
    uconv4 = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(uconv4)
    uconv4 = residual_block(uconv4,start_neurons * 16)
    uconv4 = residual_block(uconv4,start_neurons * 16, True)
    
    # 12 -> 25
    deconv3 = Conv2DTranspose(start_neurons * 8, (13, 13), strides=(1, 1), padding="valid")(uconv4)
    uconv3 = concatenate([deconv3, conv3])    
    uconv3 = Dropout(DropoutRatio)(uconv3)
    
    uconv3 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv3)
    uconv3 = residual_block(uconv3,start_neurons * 8)
    uconv3 = residual_block(uconv3,start_neurons * 8, True)

    # 25 -> 50
    deconv2 = Conv2DTranspose(start_neurons * 4, (25, 25), strides=(1, 1), padding="valid")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
        
    uconv2 = Dropout(DropoutRatio)(uconv2)
    uconv2 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv2)
    uconv2 = residual_block(uconv2,start_neurons * 4)
    uconv2 = residual_block(uconv2,start_neurons * 4, True)
    
    # 50 -> 101
    #deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="valid")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    
    uconv1 = Dropout(DropoutRatio)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv1)
    uconv1 = residual_block(uconv1,start_neurons * 1)
    uconv1 = residual_block(uconv1,start_neurons * 1, True)
    


    # 50 -> 101
    #deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    deconv0 = Conv2DTranspose(start_neurons * 1, (1, 1), strides=(2, 2), padding="valid")(uconv1)
    #uconv1 = concatenate([deconv1, conv1])
    
    uconv0 = Dropout(DropoutRatio)(deconv0 )
    uconv0 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
    uconv0 = residual_block(uconv0,start_neurons * 1)
    uconv0 = residual_block(uconv0,start_neurons * 1, True)    
    
    
    
    #uconv1 = Dropout(DropoutRatio/2)(uconv1)
    #output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    output_layer_noActi = Conv2D(1, (1,1), padding="same", activation=None)(uconv0)
    output_layer =  Activation('sigmoid')(output_layer_noActi)
    
    model = Model(base_model.input, output_layer)
    return model

In [14]:
## model
img_size_ori = 101
img_size_target = 198

# input_layer = Input((img_size_target, img_size_target, 3))
# output_layer = build_model(input_layer, 64,0.5)

input_layer = (img_size_target, img_size_target, 3)

resnet50_model = ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape= input_layer, pooling=None, )



NameError: name 'ResNet50' is not defined

In [28]:
resnet50_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 198, 198, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 99, 99, 64)   9472        input_2[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 99, 99, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_221 (Activation)     (None, 99, 99, 64)   0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [29]:
model1 = build_model(input_layer, 64,resnet50_model, 0.5)

In [30]:
model1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 198, 198, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 99, 99, 64)   9472        input_2[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 99, 99, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_221 (Activation)     (None, 99, 99, 64)   0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [32]:
model1.get_config()

{'input_layers': [['input_2', 0, 0]],
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 198, 198, 3),
    'dtype': 'float32',
    'name': 'input_2',
    'sparse': False},
   'inbound_nodes': [],
   'name': 'input_2'},
  {'class_name': 'Conv2D',
   'config': {'activation': 'linear',
    'activity_regularizer': None,
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'filters': 64,
    'kernel_constraint': None,
    'kernel_initializer': {'class_name': 'VarianceScaling',
     'config': {'distribution': 'uniform',
      'mode': 'fan_avg',
      'scale': 1.0,
      'seed': None}},
    'kernel_regularizer': None,
    'kernel_size': (7, 7),
    'name': 'conv1',
    'padding': 'same',
    'strides': (2, 2),
    'trainable': False,
    'use_bias': True},
   'inbound_nodes': [[['input_2', 0, 0, {}]]],
   'name': 'conv1'},
  

# UNET-VGG16

In [19]:
from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout, BatchNormalization, UpSampling2D,Concatenate, Activation,Add
from keras import backend as K
from keras import optimizers

from keras.applications.vgg16 import VGG16

In [12]:
vgg_model = VGG16(weights='imagenet', input_shape=(128, 128, 3), include_top=False)

In [26]:
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [5]:
# def build_model(input_layer, start_neurons, DropoutRatio = 0.5):
#     #inputs = Input(shape=(img_rows, img_cols,3))
#     conv1 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding='same')(input_layer)
#     conv1 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding='same')(conv1)
#     pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

#     conv2 = Conv2D(start_neurons * 2, (3, 3), activation='relu', padding='same')(pool1)
#     conv2 = Conv2D(start_neurons * 2, (3, 3), activation='relu', padding='same')(conv2)
#     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

#     conv3 = Conv2D(start_neurons * 4, (3, 3), activation='relu', padding='same')(pool2)
#     conv3 = Conv2D(start_neurons * 4, (3, 3), activation='relu', padding='same')(conv3)
#     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

#     conv4 = Conv2D(start_neurons * 8, (3, 3), activation='relu', padding='same')(pool3)
#     conv4 = Conv2D(start_neurons * 8, (3, 3), activation='relu', padding='same')(conv4)
#     pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

#     conv5 = Conv2D(start_neurons * 16, (3, 3), activation='relu', padding='same')(pool4)
#     conv5 = Conv2D(start_neurons * 16, (3, 3), activation='relu', padding='same')(conv5)

#     up6 = concatenate([Conv2DTranspose(start_neurons * 8, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
#     conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(up6)
#     conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)

#     up7 = concatenate([Conv2DTranspose(start_neurons * 4, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
#     conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up7)
#     conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)

#     up8 = concatenate([Conv2DTranspose(start_neurons * 2, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
#     conv8 = Conv2D(16, (3, 3), activation='relu', padding='same')(up8)
#     conv8 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv8)

#     up9 = concatenate([Conv2DTranspose(start_neurons * 1, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
#     conv9 = Conv2D(8, (3, 3), activation='relu', padding='same')(up9)
#     conv9 = Conv2D(8, (3, 3), activation='relu', padding='same')(conv9)

#     output_layer = Conv2D(3, (1, 1), activation='sigmoid')(conv9)

#     model = Model(input_layer, output_layer)

#     return model

In [10]:
## model
img_size_ori = 101
img_size_target = 128

input_layer = Input((img_size_target, img_size_target, 3))
model = build_model(input_layer, 64,0.5)

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_44 (Conv2D)              (None, 128, 128, 64) 1792        input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_45 (Conv2D)              (None, 128, 128, 64) 36928       conv2d_44[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_13 (MaxPooling2D) (None, 64, 64, 64)   0           conv2d_45[0][0]                  
__________________________________________________________________________________________________
conv2d_46 

In [45]:
#Build model for 197 input. The problem was upscale
def build_model(input_layer, start_neurons, base_model, DropoutRatio = 0.5):

    # Get Resnet
    #resnet50_model = ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape= input_layer, pooling=None)
    
    #base_model.layers.pop()
    
    for layer in base_model.layers[:19]:
        layer.trainable = False
    
    conv1 = base_model.layers[2].output
    conv2 = base_model.layers[5].output
    conv3 = base_model.layers[9].output
    conv4 = base_model.layers[13].output
    pool4 = base_model.layers[18].output
    
#     #inputs = Input(shape=(img_rows, img_cols,3))
#     conv1 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding='same')(input_layer)
#     conv1 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding='same')(conv1)
#     pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

#     conv2 = Conv2D(start_neurons * 2, (3, 3), activation='relu', padding='same')(pool1)
#     conv2 = Conv2D(start_neurons * 2, (3, 3), activation='relu', padding='same')(conv2)
#     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

#     conv3 = Conv2D(start_neurons * 4, (3, 3), activation='relu', padding='same')(pool2)
#     conv3 = Conv2D(start_neurons * 4, (3, 3), activation='relu', padding='same')(conv3)
#     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

#     conv4 = Conv2D(start_neurons * 8, (3, 3), activation='relu', padding='same')(pool3)
#     conv4 = Conv2D(start_neurons * 8, (3, 3), activation='relu', padding='same')(conv4)
#     pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(start_neurons * 16, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(start_neurons * 16, (3, 3), activation='relu', padding='same')(conv5)

    
    deconv6 = Conv2DTranspose(start_neurons * 8, (2, 2), strides=(4, 4), padding='same')(conv5)
    up6 = concatenate([deconv6, conv4])    
    conv6 = Conv2D(start_neurons * 8, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(start_neurons * 8, (3, 3), activation='relu', padding='same')(conv6)

    deconv7 = Conv2DTranspose(start_neurons * 4, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = concatenate([deconv7, conv3])  
    conv7 = Conv2D(start_neurons * 4, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(start_neurons * 4, (3, 3), activation='relu', padding='same')(conv7)

    deconv8 = Conv2DTranspose(start_neurons * 2, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = concatenate([deconv8, conv2])     
    conv8 = Conv2D(start_neurons * 2, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(start_neurons * 2, (3, 3), activation='relu', padding='same')(conv8)

    deconv9 = Conv2DTranspose(start_neurons * 1, (2, 2), strides=(2, 2), padding='same')(conv8)
    up9 = concatenate([deconv9, conv1]) 
    conv9 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding='same')(conv9)

    output_layer = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(base_model.input, output_layer)

    return model

In [31]:
## model
img_size_ori = 101
img_size_target = 128

# input_layer = Input((img_size_target, img_size_target, 3))
# output_layer = build_model(input_layer, 64,0.5)

input_layer = (img_size_target, img_size_target, 3)

vgg16_model = VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape= input_layer, pooling=None, )



In [46]:
model1 = build_model(input_layer, 64,vgg16_model, 0.5)

In [47]:
model1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 128, 128, 64) 1792        input_10[0][0]                   
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 128, 128, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 64, 64, 64)   0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [48]:
model1.get_config()


{'input_layers': [['input_10', 0, 0]],
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 128, 128, 3),
    'dtype': 'float32',
    'name': 'input_10',
    'sparse': False},
   'inbound_nodes': [],
   'name': 'input_10'},
  {'class_name': 'Conv2D',
   'config': {'activation': 'relu',
    'activity_regularizer': None,
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'filters': 64,
    'kernel_constraint': None,
    'kernel_initializer': {'class_name': 'VarianceScaling',
     'config': {'distribution': 'uniform',
      'mode': 'fan_avg',
      'scale': 1.0,
      'seed': None}},
    'kernel_regularizer': None,
    'kernel_size': (3, 3),
    'name': 'block1_conv1',
    'padding': 'same',
    'strides': (1, 1),
    'trainable': False,
    'use_bias': True},
   'inbound_nodes': [[['input_10', 0, 0, {}]]],
   'name': 'bl